## Classification of the MNIST data using CNNs

The MNIST Dataset has 28 x 28 black and white images of digits from 0 to 9. It is one of the most common datasets for starting up with deep learning. It comes in built in the keras package.

This notebook will walk you through the developing a classification model for the dataset using a <b>Convolutional Neural Network</b> with the help of the <b>Model API</b> from Keras.

### Importing required libraries

In [1]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
from matplotlib import pyplot as plt

from keras.layers import Input, Flatten, Dense, Dropout, MaxPooling2D
#from keras.layers.advanced_activations import LeakyReLU
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.convolutional import Conv2D

from keras.activations import *
from keras.optimizers import *
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.models import Model, Sequential

Using TensorFlow backend.


### Loading the dataset

MNIST comes as a part of the keras datasets. It contains 60,000 training images while 10,000 test images

In [2]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Display one image?

11501568/11490434 [==============================] - 0s 0us/step


In [3]:
X_train.shape, X_test.shape

((60000, 28, 28), (10000, 28, 28))

### Data Preprocessing

The labels refer to the expected classification output for a given image. They are converted from singular to one-hot encoded values from 0 to 9. 

That is, if a given image corresponds to 5, its encoding will be [0,0,0,0,0,1,0,0,0,0]

In [4]:
X_train = (X_train.astype(np.float32) - 127.5)/127.5
X_test = (X_test.astype(np.float32) - 127.5)/127.5

X_train = X_train[:,:,:,np.newaxis]
X_test = X_test[:,:,:,np.newaxis]

X_train.shape, X_test.shape

((60000, 28, 28, 1), (10000, 28, 28, 1))

In [0]:
num_classes = 10

# convert class vectors to binary class matrices
y_train = to_categorical(y_train, num_classes)
#y_test = to_categorical(y_test, num_classes)

### Creating a Keras Model

We will build the model using the Sequential API. Since we only wan't MLP based network, we will use Dense layers for fully connecting neurons.Simply go on adding a layer as it pleases. 

In [6]:
In = Input(shape=(28,28,1))
x = Conv2D(32, kernel_size=3, strides=2, padding="same",activation='relu')(In)
x = MaxPooling2D(pool_size=2,strides=2, padding="same")(x)
x = BatchNormalization()(x)

x = Conv2D(64, kernel_size=3, strides=2, padding="same",activation='relu')(x)
x = MaxPooling2D(pool_size=2,strides=2, padding="same")(x)
x = BatchNormalization()(x)

x = Flatten()(x)
x = Dense(64, activation='relu')(x)
y = Dense(num_classes, activation='softmax')(x)
                                                                                    

"""
model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation = "relu")
model.add(layers.Dropout(0.2, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation = "relu"))
"""                                                                                

model = Model(inputs=In,outputs=y)                                                                                   
                                                                                   
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 7, 7, 32)          128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 64)          18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 2, 2, 64)          0         
_________________________________________________________________
batc

The final model is compiled using an optimizer, a loss function and a metric for performance improvement. 
- The loss function is used to depict how far is the current model from the ideal answer
- The optimizer refers to the method that will be used to minimize the loss
- The metrics correspond to how we want to measure the performance of the network

In [0]:
model.compile(loss='categorical_crossentropy', optimizer=SGD(), metrics=['accuracy'])

### Training and testing

- Training is done using the function fit(). We train out network for 5 epochs.
- Testing is done using the predict() function. We can also use evaluate() but since we want to later generate a classificiation report, we will use the former


In [8]:
model.fit(X_train,y_train, epochs=5)

Instructions for updating:
Use tf.cast instead.
Epoch 1/5
60000/60000 [==============================] - 26s 426us/step - loss: 0.2483 - acc: 0.9285
Epoch 2/5
60000/60000 [==============================] - 25s 416us/step - loss: 0.0878 - acc: 0.9741
Epoch 3/5
60000/60000 [==============================] - 25s 415us/step - loss: 0.0649 - acc: 0.9810
Epoch 4/5
60000/60000 [==============================] - 25s 413us/step - loss: 0.0519 - acc: 0.9850
Epoch 5/5
60000/60000 [==============================] - 25s 412us/step - loss: 0.0427 - acc: 0.9877


In [9]:
y_check = model.predict(X_test)

y_pred = np.array([np.argmax(y_check[j]) for j in range(len(y_check))])
y_test

array([7, 2, 1, ..., 4, 5, 6], dtype=uint8)

In [10]:
print(confusion_matrix(y_test, y_pred))

[[ 972    0    1    0    1    1    2    1    1    1]
 [   1 1126    2    3    0    0    0    2    1    0]
 [   1    3 1018    2    0    0    0    4    4    0]
 [   0    0    4  990    0    8    0    4    1    3]
 [   0    0    1    0  964    0    4    2    1   10]
 [   0    0    1    3    0  885    1    0    2    0]
 [   6    3    2    0    3    6  935    0    3    0]
 [   0    3   10    3    0    0    0 1007    0    5]
 [   2    0    3    4    1    2    1    3  955    3]
 [   5    4    0    3    3    3    0    5    3  983]]


In [11]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.98      0.99      0.99       980
           1       0.99      0.99      0.99      1135
           2       0.98      0.99      0.98      1032
           3       0.98      0.98      0.98      1010
           4       0.99      0.98      0.99       982
           5       0.98      0.99      0.98       892
           6       0.99      0.98      0.98       958
           7       0.98      0.98      0.98      1028
           8       0.98      0.98      0.98       974
           9       0.98      0.97      0.98      1009

   micro avg       0.98      0.98      0.98     10000
   macro avg       0.98      0.98      0.98     10000
weighted avg       0.98      0.98      0.98     10000



In [0]:
model.evaluate(X_test,y_test)

ValueError: Error when checking target: expected dense_2 to have shape (10,) but got array with shape (1,)